In [1]:
!pip install git+https://github.com/Evan-Kim2028/subgraph-query-portal.git

  Cloning https://github.com/Evan-Kim2028/subgraph-query-portal.git to /tmp/pip-req-build-uy08odtt
  Running command git clone --filter=blob:none --quiet https://github.com/Evan-Kim2028/subgraph-query-portal.git /tmp/pip-req-build-uy08odtt
  Resolved https://github.com/Evan-Kim2028/subgraph-query-portal.git to commit 7993c20c8a18d0a29a86ac23c7d0f6ff13fcec3f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done

[notice] A new release of pip is available: 23.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [12]:
from queryportal.subgraphinterface import SubgraphInterface

from datetime import datetime, timedelta


import polars as pl
pl.Config.set_fmt_str_lengths(200)

polars.config.Config

In [13]:
# define subgraph endpoint. This one is the Univ3 Ethereum endpoint maintained by Messari
endpoint = 'https://api.thegraph.com/subgraphs/name/messari/uniswap-v3-ethereum'

# instantiate dex class. Dex stores dex-related query functions
my_dex = SubgraphInterface(endpoint)

First query is to get PEPE pools on Univ3

Tips:
- Find valid query entities and columns by inspecting the endpoint url....

In [26]:
pool_filter = {
    'pool_name_contains': 'Pepe',
    'timestamp_gte': int((datetime(2023, 4, 12).timestamp())),
    'timestamp_lte': int(datetime(2023, 4, 19).timestamp())
}

pool_cols = [
    'timestamp',
    'hash',
    # comes from Pool! entity
    'pool_name', 
    'pool_id',
    'pool_totalValueLockedUSD',
    # comes from Token entity
    'tokenIn_symbol', 
    'tokenOut_symbol', 
    'tokenIn_id',
    'tokenOut_id',
    ]

In [27]:
df = my_dex.query_entity(
    entity='swaps',
    name='uniswap-v3-ethereum',
    query_paths=pool_cols,
    query_size=1000,
    filter_dict=pool_filter
    )

Querying subgraph endpoint: uniswap-v3-ethereum
Shape: (1000, 9)
Schema: {'timestamp': Int64, 'hash': Utf8, 'pool_name': Utf8, 'pool_id': Utf8, 'pool_totalValueLockedUSD': Float64, 'tokenIn_symbol': Utf8, 'tokenIn_id': Utf8, 'tokenOut_symbol': Utf8, 'tokenOut_id': Utf8}
Function 'query_entity' executed in 3.4338s


In [28]:
unique_pool_df = df[['pool_name', 'pool_id', 'pool_totalValueLockedUSD']].unique().sort(by='pool_totalValueLockedUSD', descending=True)

In [29]:
unique_pool_df = unique_pool_df.to_pandas()

In [30]:
unique_pool_df

,pool_name,pool_id,pool_totalValueLockedUSD
0,Uniswap V3 McPepes/Wrapped Ether 1%,0xdf19389d0b2bee33230d73f07c8e0e2c4e78fb59,4.286022e+06
1,Uniswap V3 Pepe/Wrapped Ether 1%,0xf239009a101b6b930a527deaab6961b6e7dec8a6,2.639507e+06
2,Uniswap V3 Pepe/McPepes 1%,0x5e63add89fad07394c866b5ec650d062be15ef73,5.250808e+04
3,Uniswap V3 Pepe/USD Coin 1%,0xcee31c846cbf003f4ceb5bbd234cba03c6e940c7,4.889254e+04
4,Uniswap V3 Wojak Coin/Pepe 1%,0x0b4cb9d4fb1cee079392bc7cf6417a669f4c2dbd,3.448001e+02


In [9]:
# it appears that the main pool liquidity for PEPE is this pool_id 0xf239009a101b6b930a527deaab6961b6e7dec8a6

In [31]:
swap_filter = {
    'timestamp_gte': int((datetime(2023, 4, 19) - timedelta(days=7)).timestamp()),
    'timestamp_lte': int(datetime(2023, 4, 19).timestamp()),
    'pool_id': '0xf239009a101b6b930a527deaab6961b6e7dec8a6'
}

swap_cols = [
    'account_id',
    'tokenIn_symbol', 
    'tokenOut_symbol', 
    'amountOutUSD', 
    'amountInUSD',
    'amountOut',
    'amountIn'
    ]

swap_df = my_dex.query_entity(
    entity='swaps',
    name='uniswap-v3-ethereum',
    query_paths=swap_cols, 
    filter_dict=swap_filter, 
    query_size=100, 
    saved_file_name='pepe_swaps_4.12.23_to_4.19.23')


Querying subgraph endpoint: uniswap-v3-ethereum
Shape: (100, 7)
Schema: {'amountOutUSD': Float64, 'amountInUSD': Float64, 'amountOut': Float64, 'amountIn': Float64, 'account_id': Utf8, 'tokenIn_symbol': Utf8, 'tokenOut_symbol': Utf8}
Function 'query_entity' executed in 1.0917s


In [32]:
swap_df

amountOutUSD,amountInUSD,amountOut,amountIn,account_id,tokenIn_symbol,tokenOut_symbol
f64,f64,f64,f64,str,str,str
1813.988391,1882.034695,8.5473e27,9.0000e17,"""0x48cbfe2b07c4aa6ac75b13d5465b9b2f2e43dd0f""","""WETH""","""PEPE"""
0.0,154.197149,4.2686e27,7.4300e16,"""0xb12ea163a895829112798b1713c8f1d2a4821025""","""WETH""","""PEPE"""
0.0,83.666231,1.3472e27,4.0000e16,"""0x477d7c3218bed2b72a451178689785674410aeb5""","""WETH""","""PEPE"""
147.381486,147.082163,8.9104e26,7.0000e16,"""0xd0885451654c9d6c5017582d544e9916f86a245d""","""WETH""","""PEPE"""
0.0,60.76173,1.0564e27,2.9000e16,"""0xaa525d002e0240a9c6461e3400e37962e4f0c5bc""","""WETH""","""PEPE"""
276.008582,278.993839,1.5230e27,1.3400e17,"""0x86b1810e93e9f76b8157934eeb18db163ef40bfc""","""WETH""","""PEPE"""
0.0,114.123993,1.7806e27,5.5000e16,"""0xf8ed350ce552f7d54652eed4b47daed93146d13d""","""WETH""","""PEPE"""
4782.695608,4767.987232,2.2814e18,2.7500e28,"""0x468d1ef7a1204d592537dab35794aac640c97505""","""PEPE""","""WETH"""
0.0,166.554623,2.8962e27,8.0000e16,"""0x13dc7217d233e51d8c724052139b7afd29bb57e5""","""WETH""","""PEPE"""
